In [2]:
import os
import pickle
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import tree


In [4]:
dataset = pd.read_csv('03_data.csv')
#View all the columns
pd.set_option('display.max_columns', None)
dataset.head()

,Unnamed: 0,precio_m2,superficie_total,categoria,dormitorios,baños,cocheras,colonia,alcaldia,dist_metro
0,0,60144.927536,138.0,departamento,2.0,3.0,2.0,POLANCO REFORMA (POLANCO),MIGUEL HIDALGO,499.558324
1,1,104651.162791,172.0,departamento,2.0,2.0,2.0,POLANCO REFORMA (POLANCO),MIGUEL HIDALGO,755.632370
2,2,85919.540230,174.0,departamento,2.0,2.0,2.0,POLANCO REFORMA (POLANCO),MIGUEL HIDALGO,772.890462
3,3,5092.622951,183.0,departamento,3.0,3.0,2.0,POLANCO REFORMA (POLANCO),MIGUEL HIDALGO,1004.848305
4,4,93125.000000,160.0,departamento,2.0,2.0,2.0,POLANCO REFORMA (POLANCO),MIGUEL HIDALGO,480.029999


In [5]:
df=dataset.dropna(how='any')
df.count()

Unnamed: 0          6166
precio_m2           6166
superficie_total    6166
categoria           6166
dormitorios         6166
baños               6166
cocheras            6166
colonia             6166
alcaldia            6166
dist_metro          6166
dtype: int64

In [8]:
df['colonia'].unique()


array(['POLANCO REFORMA (POLANCO)', 'BOSQUES DE LAS LOMAS',
       'PALO ALTO (GRANJAS)', 'LOS CIPRESES', 'LOS MORALES (POLANCO)',
       'CONDESA', 'MORALES SECCION ALAMEDA (POLANCO)',
       'JARDINES DE LA PALMA (HUIZACHITO)', 'GRANADA (AMPL)',
       'LOMAS DE BEZARES', 'OLIVAR DE LOS PADRES', 'TLACOPAC',
       'SANTA MARIA LA RIBERA I',
       'LOMAS DE BARRILACO (LOMAS DE CHAPULTEPEC)',
       'SANTO DOMINGO (PBLO)', 'MORALES SECCION PALMAS (POLANCO)',
       'LOMAS DEL CHAMIZAL', 'HIPODROMO I', 'SAN PEDRO DE LOS  PINOS',
       'SAN JERONIMO ACULCO - LIDICE (PBLO)', 'JARDINES DEL PEDREGAL',
       'LA OTRA BANDA', 'LA LOMA', 'VERTIZ NARVARTE',
       'LOMAS DE SANTA FE II', 'IRRIGACION', 'SAN ANGEL', 'SAN JUAN',
       'LOMAS DE REFORMA (LOMAS DE CHAPULTEPEC)',
       'BOSQUES DE CHAPULTEPEC (POLANCO)', 'DEL VALLE VI',
       'LOMAS DE CHAPULTEPEC', 'FUENTES DEL PEDREGAL', 'TABACALERA',
       'LOMAS DE VISTA HERMOSA', 'TIZAPAN_TIZAPAN SAN ANGEL',
       'CTM VI CULHUACAN (U HA

In [11]:
target = df["colonia"]
target_names = ['POLANCO REFORMA (POLANCO)', 'BOSQUES DE LAS LOMAS',
       'PALO ALTO (GRANJAS)', 'LOS CIPRESES', 'LOS MORALES (POLANCO)',
       'CONDESA', 'MORALES SECCION ALAMEDA (POLANCO)',
       'JARDINES DE LA PALMA (HUIZACHITO)', 'GRANADA (AMPL)',
       'LOMAS DE BEZARES', 'OLIVAR DE LOS PADRES', 'TLACOPAC',
       'SANTA MARIA LA RIBERA I',
       'LOMAS DE BARRILACO (LOMAS DE CHAPULTEPEC)',
       'SANTO DOMINGO (PBLO)', 'MORALES SECCION PALMAS (POLANCO)',
       'LOMAS DEL CHAMIZAL', 'HIPODROMO I', 'SAN PEDRO DE LOS  PINOS',
       'SAN JERONIMO ACULCO - LIDICE (PBLO)', 'JARDINES DEL PEDREGAL',
       'LA OTRA BANDA', 'LA LOMA', 'VERTIZ NARVARTE',
       'LOMAS DE SANTA FE II', 'IRRIGACION', 'SAN ANGEL', 'SAN JUAN',
       'LOMAS DE REFORMA (LOMAS DE CHAPULTEPEC)',
       'BOSQUES DE CHAPULTEPEC (POLANCO)', 'DEL VALLE VI',
       'LOMAS DE CHAPULTEPEC', 'FUENTES DEL PEDREGAL', 'TABACALERA',
       'LOMAS DE VISTA HERMOSA', 'TIZAPAN_TIZAPAN SAN ANGEL',
       'CTM VI CULHUACAN (U HAB)', 'LOMAS DE AXOMIATLA',
       'CHAPULTEPEC MORALES (POLANCO)', 'TLALPAN CENTRO',
       'PRADO COAPA 2A SECCION', 'CORREDOR SANTA FE',
       'SAN BARTOLO AMEYALCO (PBLO)', 'SAN NICOLAS TOTOLAPAN (PBLO)',
       'EXTREMADURA INSURGENTES', 'NARVARTE V', 'PERIODISTA',
       'DEL VALLE III', 'CHAPULTEPEC POLANCO (POLANCO)', 'ANAHUAC II',
       'NARVARTE II', 'LAS CUEVITAS', 'LINDAVISTA II', 'DEL VALLE V',
       'PASEO DE LAS LOMAS-SAN GABRIEL', 'NARVARTE III',
       'SANTA MARIA NONOALCO', 'NUEVA ORIENTAL COAPA-EX HACIENDA COAPA',
       'BOSQUES DE TARANGO', 'ROMERO DE TERREROS (FRACC)',
       'SANTA CATARINA (BARR)', 'JARDINES DE COYOACAN (FRACC)',
       'GRAL C  A  MADRAZO', 'ANAHUAC LOS MANZANOS', 'ROMA SUR II',
       'CLUB DE GOLF MEXICO-SAN BUENAVENTURA', 'LOMAS DE REFORMA',
       'JARDINES DEL AJUSCO', 'EX HACIENDA COAPA', 'XOTEPINGO',
       'HIPODROMO II', 'ROMA NORTE III', 'MANZANASTITLA',
       'OXTOPULCO UNIVERSIDAD', 'CUAUHTEMOC', 'JARDIN AZPEITIA',
       'NAPOLES (AMPL)', 'HIPODROMO CONDESA', 'JARDINES EN LA MONTAA',
       'PARQUES DEL PEDREGAL', 'DEL VALLE II', 'EL CONTADERO', 'LEGARIA',
       'CENTRO URBANO PRESIDENTE ALEMAN (U HAB)', 'SAN JOSE INSURGENTES',
       'MIGUEL HIDALGO', 'COPILCO EL BAJO', 'LAS AGUILAS (AMPL)',
       'SAN MATEO TLALTENANGO (PBLO)', 'GRANADA', 'ABDIAS GARCIA SOTO',
       'JOYAS DEL PEDREGAL (FRACC)',
       'LOMAS VIRREYES (LOMAS DE CHAPULTEPEC)', 'DEL CARMEN',
       'NAVIDAD (GRANJAS DE NAVIDAD)', 'TETELPAN', 'ESCANDON II',
       'SAN PEDRO XALPA (PBLO)', 'AMPLIACION MIGUEL HIDALGO 2A SECC',
       'NOCHE BUENA', 'DEL VALLE IV', 'NONOALCO', 'NARVARTE I', 'FLORIDA',
       'NAPOLES', 'ALAMOS I', 'LAS AGUILAS', 'TETELPAN (PBLO)', 'ALPES',
       'PUENTE COLORADO', 'DEL VALLE VII', 'EL RELOJ', 'PORTALES III',
       'INSURGENTES CUICUILCO',
       'BOSQUE DE CHAPULTEPEC I, II Y III SECCIONES', 'ALPES (AMPL)',
       'CIUDAD DE LOS DEPORTES', 'PASEOS DE CHURUBUSCO',
       'SAN LUCAS (BARR)', 'ART 4TO CONSTITUCIONAL (U HAB)',
       'ALFONSO XIII', 'PALMITAS (POLANCO)', 'ALAMOS II',
       'REFORMA SOCIAL', 'LA MALINCHE', 'TORRES SAN ANTONIO (U HAB)',
       'SAN PEDRO CUAJIMALPA (PBLO)', 'DEL BOSQUE (POLANCO)', 'JUAREZ',
       'VALLEJO PONIENTE', 'FUEGO NUEVO', 'EL PARQUE DE COYOACAN (FRACC)',
       'SAN CLEMENTE', 'DEL VALLE I', 'EL YAQUI', 'SAN SIMON TICUMAC',
       'SAN IGNACIO (BARR)', 'PANTITLAN V',
       'SAN JUAN DE ARAGON 3A SECCION (U HAB) II',
       'PEDREGAL DE SAN ANGEL (AMPL)', 'CORPUS CHRISTY', 'NUEVA VALLEJO',
       'MARIA DEL CARMEN', 'LETRAN VALLE', 'TORIELLO GUERRA',
       'AGUILAS PILARES', 'TORRES TOREO', 'ROMA SUR I', 'ROMA NORTE I',
       'PORTALES IV', 'ACTIPAN', 'NARVARTE IV', 'PORTALES II',
       'SEARS ROEBUCK (U HAB)', 'ANZURES', 'SAN RAFAEL I',
       'PANTITLAN III', 'BUENAVISTA II', 'CHIMALISTAC', 'TACUBA',
       'OCHO DE AGOSTO', 'SAN ANGEL INN', 'ACACIAS', 'STA CRUZ ATOYAC',
       'PRADO CHURUBUSCO', 'CUADRANTE DE SAN FRANCISCO',
       'LOMAS QUEBRADAS', 'LOMAS DE MEMETLA', 'MODELO (U)', 'LA FAMA',
       'HOGAR Y SEGURIDAD/NUEVA SANTA MARIA', 'DEL GAS', 'ATLAMPA',
       'PORTALES ORIENTE', 'SANTA MARIA LA RIBERA IV',
       'PRESIDENTES EJIDALES PRIMERA SECCION', 'VALLE DE TEPEPAN',
       'TLACOQUEMECATL DEL VALLE', 'PARQUE SAN ANDRES', 'PORTALES I',
       'BUENAVISTA I', 'CUAUHTEMOC PENSIL', 'FLORESTA-PRADO-VERGEL COAPA',
       'SANTA MARIA AZTAHUACAN (U HAB)', 'ROMA NORTE II', 'GUADALUPE INN',
       'INTEGRACION LATINOAMERICANA (U HAB)', 'IMAN 580 (U HAB)',
       'GRANJAS MODERNAS-SAN JUAN DE ARAGON (AMPL)', 'OBRERA III', 'XOCO',
       'RINCON DEL BOSQUE', 'VERONICA ANZURES', 'PROGRESISTA',
       'ARENAL PUERTA TEPEPAN', 'PRESIDENTES EJIDALES SEGUNDA SECCION',
       'LOMAS HIDALGO', 'LAS TINAJAS', 'CIUDAD UNIVERSITARIA',
       'SANTIAGO SUR', 'TEPEYAC INSURGENTES', 'LOS GIRASOLES III',
       'ATLAMAYA', 'ALIANZA POPULAR REVOLUCIONARIA ORIENTE (U HAB)',
       'MIXCOAC', 'HEROES DE PADIERNA', 'NARVARTE VI', 'DOCTORES II',
       'CONJUNTO HABITACIONAL PEDREGAL DEL LAGO',
       'SAN PEDRO MARTIR (PBLO)', 'EX EJIDO SAN FRANCISCO CULHUACAN I',
       'NARCISO MENDOZA VILLA COAPA SUPERMANZANA 8 (U HAB)',
       'LOMAS DE TETELPAN', 'PEDREGAL DE LA ZORRA', 'GRANJAS SAN ANTONIO',
       'GARCIMARRERO NORTE', 'EL VERGEL', 'LOMAS ESTRELLA 2A SECC II',
       'SAN ANDRES TOTOLTEPEC (PBLO)', 'LA DRAGA', 'LOMAS DE GUADALUPE',
       'GENERAL PEDRO MARIA ANAYA', 'LINDAVISTA I', 'NATIVITAS',
       'LOMAS DE SANTA FE', 'AMPLIACION PETROLERA', 'LOS REYES (PBLO)',
       'AMPLIACION MIGUEL HIDALGO 3A SECC', 'PORTAL DEL SOL',
       'PROGRESO TIZAPAN', 'NIO JESUS (BARR)', 'LOS REYES',
       'SAN LUCAS XOCHIMANCA (PBLO)', 'SANTA URSULA COAPA (PBLO)',
       'CONSTITUCION DE LA REPUBLICA', 'REAL DEL MORAL (FRACC)',
       'CTM IXA CULHUACAN (U HAB)',
       'CONDOMINIO DEL BOSQUE (FRACC)-BOSQUE DE TLALPAN',
       'LOMAS DE SANTA CRUZ MEYEHUALCO', 'FUENTES Y PEDREGAL DE TEPEPAN',
       'CUITLHUAC 3 Y 4 (U HAB)', 'ATLANTIDA', 'HEROES DE PADIERNA II',
       'RANCHO EL ROSARIO', 'PASEOS DE TAXQUEA I',
       'SANTA MARIA LA RIBERA II', 'LOMAS ALTAS', 'ATACAXCO',
       'SAN JUAN TLIHUACA (PBLO)', 'TRABAJADORES DEL HIERRO',
       'DEFENSORES DE LA REPUBLICA', 'ASTURIAS', 'SAN RAFAEL II',
       'PIEDAD NARVARTE', 'SN JUAN TEPEXIMILPA', 'CAMPESTRE CHURUBUSCO',
       'CLAVERIA', 'SANTA LUCIA (BARR)', 'TACUBAYA', 'DEL MAR NORTE',
       'LA CONCEPCION (BARR)', 'LA NORIA TEPEPAN',
       'SAN  MIGUEL AMANTLA (PBLO)', 'LOMAS DE LAS AGUILAS',
       'COLINAS DEL BOSQUE-LAS TORTOLAS', 'NIOS HEROES DE CHAPULTEPEC',
       'LA PRIMAVERA', 'JARDINES DEL SUR', 'CTM VIII CULHUACAN (U HAB)',
       'RESIDENCIAL ZACATENCO', 'VILLA PANAMERICANA 1ERA. SECCIN (U HAB)',
       'SANTA MARIA AZTAHUACAN (EJ) I', 'ACUEDUCTO DE GUADALUPE (RDCIAL)',
       'LAS AMERICAS (U HAB)', 'DOCTORES V', 'ANAHUAC I',
       'PEDREGAL DE SN NICOLAS 3A SECC', 'BARRIO 18',
       'SAN FRANCISCO CULHUACAN (PBLO)',
       'PEDREGAL DE SN NICOLAS 4A SECC II', 'TORRE BLANCA (AMPL)',
       'INDEPENDENCIA', 'PASEOS DE TAXQUEA II',
       'FUENTES BROTANTES MIGUEL HIDALGO (U HAB)',
       'LOMAS DE LOS ANGELES TETELPAN', 'GRANJAS COAPA', 'MODERNA',
       'INDUSTRIAL I', 'PETROLERA', 'SAN MIGUEL TOPILEJO (PBLO)',
       'MINAS DE CRISTO', 'HACIENDA DE GUADALUPE CHIMALISTAC',
       'OLIVAR DEL CONDE 2DA SECCION I', 'LA CANDELARIA TICOMAN (BARR)',
       'INDEPENDENCIA BATAN SUR (U HAB)', 'RICARDO FLORES MAGON',
       'MOCTEZUMA 2A SECCION I', 'CRUZ BLANCA', 'CENTRO IV',
       'GRANJAS MEXICO I', 'MILITAR MARTE', 'LA NORIA', 'EL SIFON',
       'VALLE ESCONDIDO', 'SANTA URSULA XITLA', 'SAN JUAN TEPEPAN',
       'EL ROSEDAL II', 'REFORMA IZTACCIHUATL NORTE',
       'MESA LOS HORNOS, TEXCALTENCO', 'ROMERO DE TERREROS', 'TEZOZOMOC',
       'XALTOCAN (BARR)', 'MERCED GOMEZ', 'TLACAMACA', 'DOCTORES IV',
       'VILLA VERDUN', 'HACIENDA SAN JUAN-RINCON DE SAN JUAN-CHIMALI',
       'SANTA CECILIA', 'JORGE NEGRETE', 'GUADALUPE TEPEYAC',
       'FUENTES DE ZARAGOZA (U HAB)', 'SAN ALVARO', 'SIETE MARAVILLAS',
       'GRANJAS COAPA ORIENTE', 'BARRANCA SECA', 'AGRICOLA ORIENTAL IV',
       'CASA BLANCA', 'ESTRELLA', 'COPILCO UNIVERSIDAD', 'ALBERT',
       'LA MARTINICA', 'TRANSITO', 'NONOALCO-TLATELOLCO (U HAB) II',
       'AXOTLA', 'VILLA PANAMERICANA 2DA. SECCIN (U HAB)',
       'SAN NICOLAS TOLENTINO II', 'INSURGENTES MIXCOAC',
       'LOMAS DE TARANGO', 'EL CUERNITO', 'SAN CRISTOBAL (BARR)',
       'TIZAMPAMPANO', 'BARROS SIERRA', 'AVIACION CIVIL (AMPL)',
       'LA PALMITA', 'SAN DIEGO CHURUBUSCO', 'GUADALUPE VICTORIA II',
       'DANIEL GARZA (AMPL)', 'NICOLAS BRAVO', 'NUEVA ATZACOALCO III',
       'JARDIN BALBUENA I', 'VILLA PANAMERICANA 4TA. SECCIN (U HAB)',
       'LA  LONJA', 'ALCANTARILLA', 'JARDIN BALBUENA III', 'SANTA ANITA',
       'LOMAS DE  PADIERNA I', 'LAS HUERTAS', 'MORELOS I', 'EL MIRADOR',
       'RESIDENCIAL EMPERADORES', 'MAGDALENA ATLAZOLPA (PBLO)',
       'AHUATENCO', 'SAN LORENZO HUIPULCO', 'SANTA URSULA COYOACAN',
       'BANJIDAL', 'DEL GAS (AMPL)', 'EL PRADO', 'COUNTRY CLUB',
       'PASTEROS', 'TORRES LINDAVISTA (FRACC)', 'LOMAS DE  PADIERNA II',
       '7 DE NOVIEMBRE', 'AJUSCO II', 'EL TRIUNFO', 'CANTIL DEL PEDREGAL',
       'VALLEJO II', 'REFORMA PENSIL',
       'REAL DEL SUR-VILLAS DEL SUR-RESIDENCIAL ACOXPA',
       'INSURGENTES SAN BORJA', 'RENOVACION', 'JARDIN BALBUENA II',
       'EX EJIDOS DE LA MAGDALENA MIXIHUCA', 'TORRES DE POTRERO',
       'JUVENTUD UNIDA', 'HEROES DE PADIERNA I', 'VALLEJO I',
       'PEDREGAL DE STO DOMINGO III', 'LOMAS DE SAN ANGEL INN',
       'LOMAS DE SOTELO', 'ANAHUAC LAGO SUR', 'ESCANDON I',
       'SAN MIGUEL CHAPULTEPEC II', 'EMANCIPACION DEL PUEBLO',
       'VIADUCTO PIEDAD', 'SAN BARTOLO CAHUALTONGO (PBLO)', 'OLIMPICA',
       'REFORMA IZTACCIHUATL SUR', 'AGRICOLA ORIENTAL V',
       'NUEVA STA ANITA', 'GUADALUPE INSURGENTES',
       'TORRES SAN PEDRO (U HAB)', 'EL JAGUEY-ESTACION PANTACO',
       'TLAPECHICO', 'PEDREGAL DE LAS AGUILAS',
       'SAN PABLO TEPETLAPA (PBLO)', 'ESPARTACO',
       'LA TORTUGA XOLALPA-HCO COLEGIO MILITAR',
       'ARENAL GUADALUPE TLALPAN', 'OCOTILLOS', 'PENSIL SAN JUANICO',
       'SAN JUAN DE ARAGON (PBLO)', 'SAN ANTONIO (FRACC)',
       'PARQUE RESIDENCIAL SAN ANTONIO (U HAB)',
       'MAGDALENA  MIXHUCA (PBLO)', 'SECTOR NAVAL',
       'SANTA MARIA TEPEPAN (PBLO)', 'FERRERIA (U HAB)', 'AGUILERA',
       'COLINAS DEL SUR', 'TORRE BLANCA', 'PENSIL NORTE',
       'VILLA COYOACAN', 'TEPEPAN (AMPL)', 'SAN JOSE DE LOS CEDROS II',
       'ZACAHUITZCO', 'VILLA QUIETUD (FRACC)', 'VILLA DE ARAGON (FRACC)',
       'CANTERA PUENTE DE PIEDRA', 'LOMA HERMOSA (CONJ HAB)',
       'ASTURIAS (AMPL)', 'SAN JUAN XALPA II', 'GUADALUPE PROLETARIA',
       'ARGENTINA ANTIGUA', 'SANTIAGO NORTE',
       'PEDREGAL DE SAN FRANCISCO (FRACC)', 'OBRERO POPULAR',
       'BOSQUES DE TETLAMEYA', 'SAN SIMON TOLNAHUAC',
       'LA PURISIMA TICOMAN (BARR)',
       'AMPLIACION SAN FRANCISCO CULHUACAN (EJ)', 'IMAN', 'SAN JOAQUIN',
       '2A AMPLIACION SANTIAGO ACAHUALTEPEC I',
       'SANTA ISABEL TOLA (PBLO)', 'BELEN DE LAS FLORES_', 'EL MOLINO',
       'AMERICAS UNIDAS-DEL LAGO', 'RESIDENCIAL MARIA ISABEL',
       'ACULCO (PBLO)', 'PUEBLO NUEVO BAJO', 'LA JOYA',
       'SALVADOR DIAZ MIRON', 'HUICHAPAN', 'HEROES DE CHURUBUSCO',
       'SAN PEDRO DE LOS PINOS', 'POPOTLA II', 'CENTRO VIII',
       'SAN JUAN DE ARAGON 6A SECCION (U HAB) II',
       'VERGEL DE COYOACAN-VERGEL DEL SUR', 'PASEOS DEL SUR',
       'PRESIDENTES DE MEXICO', 'TABLAS DE SAN AGUSTIN',
       'NARCISO MENDOZA-VILLA COAPA SUPER MANZANA 7 (U HAB)',
       'NUEVA SANTA MARIA', 'PALMATITLA', 'NUEVA ATZACOALCO II',
       'MOVIMIENTO ORGANIZADO DE TLALPAN-EL MIRADOR (RNCDA)',
       'SAN SALVADOR CUAUHTENCO (PBLO)', 'EDUCACION',
       'EL ARENAL 4A SECCION', 'TARANGO (U HAB)', 'HERMOSILLO',
       'DEL MAESTRO', 'LOMAS DE ZARAGOZA', 'TLACOTAL RAMOS MILLAN',
       'JUSTO SIERRA', 'LOS ANGELES', 'AVANTE', 'MORELOS II',
       'PEDREGAL DE STO DOMINGO VI', 'LA NUEVA ROSITA',
       'SAN GREGORIO ATLAPULCO (PBLO)', 'COAPA-VILLA CUEMANCO',
       'GERTRUDIS SANCHEZ 2A SECCION', 'ATENOR SALAS',
       'PICOS IZTACALCO 1B', 'PROVIDENCIA I', 'LA ASUNCION (BARR)',
       'AGRICOLA ORIENTAL III', 'VILLA GUSTAVO A MADERO',
       'SAN JUAN DE ARAGON 4A  Y 5A SECCION (U HAB) II', 'SAN LORENZO',
       'SECTOR POPULAR', 'PEDREGAL DE SN NICOLAS 1A SECC',
       'SAN BERNABE OCOTEPEC (PBLO)', 'LOS GIRASOLES I',
       'SAN LORENZO ACOPILCO (PBLO)', 'MARTIRES DE RIO BLANCO',
       'LA MAGDALENA CULHUACAN (BARR)',
       'EJIDOS DE SAN PEDRO MARTIR I (NORTE)', 'LOMAS DE SAN LORENZO II',
       'SAN MATEO (BARR)', 'RAMOS MILLAN BRAMADERO I',
       'PEDREGAL DE SANTA URSULA III', 'SAN NICOLAS TOTOLAPAN',
       'PRADOS DE COYOACAN', 'JOSEFA ORTIZ DE DOMINGUEZ',
       'SAN JUAN DE ARAGON 2A SECCION (U HAB) II',
       'VIEJO EJIDO SANTA URSULA COAPA', 'SANTA MARTHA ACATITLA_(PBLO)',
       'INDEPENDENCIA SAN RAMON (U HAB)', 'GUERRERO III', 'SAN FRANCISCO',
       'EL PARQUE', 'DE LA CANDELARIA (PBLO)',
       'SAN LORENZO ATEMOAYA (PBLO)', 'SAN JUAN XALPA I', 'GOLONDRINAS',
       'SAN BERNABE (BARR)', 'PROGRESO NACIONAL (AMPL)', 'SINATEL (AMPL)',
       'CREDITO CONSTRUCTOR', 'JACARANDAS', 'CAMPESTRE ARAGON II',
       'EJIDOS DE SAN PEDRO MARTIR II (SUR)',
       'BOSQUE RESIDENCIAL DEL SUR (FRACC)', '15 DE AGOSTO',
       'PLAZUELA DEL PEDREGAL', 'RINCON DE LA BOLSA',
       'LA MAGDALENA ATLITIC (PBLO)', 'SANTA FE',
       'MAGDALENA DE LAS SALINAS', 'QUIAHUATLA',
       'LA LAGUNA TICOMAN (BARR)', 'JUAN ESCUTIA I', 'PROVIDENCIA III',
       'VILLA COAPA (RDCIAL)', 'AGRICULTURA', 'COLINAS DE TARANGO',
       'VILLA OLIMPICA LIBERADOR MIGUEL HIDALGO (U HAB)', 'DEL RECREO',
       'PORVENIR', 'PRO HOGAR I', 'CONSTITUCION DE 1917 II', 'LA PRADERA',
       'SANTA ROSA XOCHIAC (PBLO)',
       'ROMULO SANCHEZ-SAN FERNANDO (BARR)-PEA POBRE', 'ALGARIN',
       'SAN FRANCISCO (BARR)', 'PALMITAS', 'ARBOLEDAS DEL SUR',
       'SANTA MARTHA ACATITLA', 'ALDANA', 'SANTA CRUZ XOCHITEPEC (PBLO)',
       'NATIVITAS LA JOYA (AMPL)', 'SAN BARTOLO ATEPEHUACAN (PBLO)',
       'PEDREGAL DE SANTA URSULA IV', 'VALLE DEL TEPEYAC',
       'PLANETARIO LINDAVISTA', 'PEDREGAL DE SN NICOLAS 2A SECC',
       'AMPLIACION MIGUEL HIDALGO 4A SECC', 'PUEBLO NUEVO ALTO',
       'LOMAS ESTRELLA 1A SECC (FRACC)', 'JALALPA (AMPL)',
       'MA ESTHER ZUNO DE ECHEVERRIA-TLALPUENTE', 'BOSQUES DEL PEDREGAL',
       'SAN MARCOS (AMPL)', 'EX-ESCUELA DE TIRO',
       'ACUEDUCTO DE GUADALUPE (U HAB)', 'VALENTIN GOMEZ FARIAS',
       'ARENAL', 'ZENON DELGADO', 'LOMAS DE CENTENARIO (U HAB)',
       'TRES ESTRELLAS', 'KENNEDY (U HAB)', 'MOCTEZUMA 2A SECCION II',
       'LA HERRADURA', 'PROF JOSE ARTURO LOPEZ', 'PROVIDENCIA (AMPL)',
       'SAN FELIPE DE JESUS IV', 'SAN PEDRO EL CHICO', 'SANTA FE (U HAB)',
       'GENERAL FELIPE BERRIOZABAL', 'BARRIO NORTE', 'IGNACIO ZARAGOZA I',
       'LAS AGUILAS SECC HORNOS', 'DANIEL GARZA', 'AGUA BENDITA',
       'C T M ARAGON (U)', 'MOLINO DE SANTO DOMINGO',
       '5TO TRAMO DE 20 DE NOVIEMBRE', 'ADOLFO LOPEZ MATEOS',
       'GERTRUDIS SANCHEZ 1A SECCION', 'ARCOS DE CENTENARIO',
       'SAN FELIPE DE JESUS III', 'SAN MIGUEL CHAPULTEPEC I',
       'SAN JOSE DE LOS CEDROS I', 'LA CASILDA', 'CUAUTEPEC DE MADERO',
       'SAN DIEGO OCOYOACAC', 'CTM ATZACOALCO (U HAB)', 'POPULAR RASTRO',
       'SAN JUAN DE ARAGON 2A SECCION (U HAB) I', 'FLOR DE MARIA',
       'JESUS DEL MONTE', 'EMILIANO ZAPATA (AMPL)', 'LA CAADA',
       'MOCTEZUMA 1A SECCION', 'MOCTEZUMA 2A SECCION III',
       'SANTIAGO ATEPETLAC', 'SAN JUAN DE ARAGON 1A SECCION (U HAB) I',
       'SAN JOSE DE LA ESCALERA', 'PERIODISTA FRANCISCO ZARCO',
       'LA FORESTAL 2', 'LOMAS DE CAPULA', 'GUADALUPE PROLETARIA (AMPL)',
       'EL CAPULIN', 'NUEVA TENOCHTITLAN', 'LA ESMERALDA (U HAB)',
       'CUCHILLA DEL TESORO', 'ZONA ESCOLAR I',
       'EX HIPODROMO DE PERALVILLO', 'NARCISO BASSOLS (U HAB)', 'EL TORO',
       'SAN PEDRO ZACATENCO (PBLO)', 'CASAS ALEMAN (AMPL) I',
       'AMPL CARACOL', 'SAN JUAN DE ARAGON 1A SECCION (U HAB) II',
       'FELIPE ANGELES', 'EL CARMEN', 'JOSE MARIA PINO SUAREZ',
       'CHURUBUSCO TEPEYAC',
       'SAN JUAN DE ARAGON 4A  Y 5A SECCION (U HAB) I', 'SANTA ROSA',
       'MAGDALENA  MIXHUCA', 'ZONA ESCOLAR II', '20 DE NOVIEMBRE',
       'SANTA LUCIA', 'SAN JOSE TICOMAN', 'TRIUNFO DE LA REPUBLICA',
       'SAN FELIPE DE JESUS I', 'LOMAS DE PUERTA GRANDE',
       'EL COYOL (U HAB)', 'JALALPA EL GRANDE', 'CASAS ALEMAN (AMPL) II',
       'ANAHUAC MARIANO ESCOBEDO',
       'SAN JUAN DE ARAGON 6A SECCION (U HAB) I', 'PUEBLA', 'ACUEDUCTO',
       'COVE', 'VALLE DE MADERO', 'MARTIN CARRERA I', 'POPOTLA I',
       'C T M ARAGON AMPLIACION (U)', 'VASCO DE QUIROGA', 'OBSERVATORIO',
       '7 DE JULIO', 'LORENZO BOTURINI', 'EL PIRUL', 'PRIVADA CENTENARIO',
       'IZTACCIHUATL', 'POSTAL', 'VILLA DE CORTES', 'MICHOACANA (AMPL)',
       'SIMON  BOLIVAR', 'TOLTECA', 'AMERICA', 'AMADO NERVO',
       'LA PRADERA II (U HAB)', 'COOPERATIVA CEHUAYO', 'LA CRUZ',
       'OLIVAR DEL CONDE 1RA SECCION II', 'VENUSTIANO CARRANZA',
       'LOMAS DE LA ERA', 'LOS GIRASOLES II', 'MALACATES (AMPL)']
      

In [16]:
data = df.drop(["colonia","categoria", "alcaldia"], axis=1)
feature_names = data.columns
data.head()

,Unnamed: 0,precio_m2,superficie_total,dormitorios,baños,cocheras,dist_metro
0,0,60144.927536,138.0,2.0,3.0,2.0,499.558324
1,1,104651.162791,172.0,2.0,2.0,2.0,755.632370
2,2,85919.540230,174.0,2.0,2.0,2.0,772.890462
3,3,5092.622951,183.0,3.0,3.0,2.0,1004.848305
4,4,93125.000000,160.0,2.0,2.0,2.0,480.029999


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [18]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8780804150453956

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.7723735408560312

In [20]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.34553806031649, 'Unnamed: 0'),
 (0.2266002075413546, 'dist_metro'),
 (0.14648203205233348, 'superficie_total'),
 (0.14473921222740574, 'precio_m2'),
 (0.04994330984291558, 'cocheras'),
 (0.04573226178925205, 'baños'),
 (0.040964916230248526, 'dormitorios')]

In [22]:
from tensorflow.keras.models import Sequential

model = Sequential()

In [23]:
from tensorflow.keras.layers import Dense
number_inputs = 3
number_hidden_nodes = 4
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

In [24]:
number_classes = 2
model.add(Dense(units=number_classes, activation='softmax'))

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 16        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
Total params: 26
Trainable params: 26
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Use categorical crossentropy for categorical data and mean squared error for regression
# Hint: your output layer in this example is using software for logistic regression (categorical)
# If your output layer activation was `linear` then you may want to use `mse` for loss
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
# Save the model
model.save("voice_model_trained.h5")